# How are direct fits of Moments to Thrown Monte Carlo Performing?
This notebook will quickly explore whether our fits with moments are performing well, but analyzing the outcome of 100 randomized fits, each with 30 unbounded moment parameters.

## Loading in Data

We could create a separate `best_fit.csv` file, but its simpler to quickly just know that `omegapi_23.fit` produced the best likelihood, and so we'll be using that one as the "best" comparison

In [ ]:
import pandas as pd

df = pd.read_csv("rand_moments.csv")
df_cov = pd.read_csv("rand_moments_cov.csv")
df_corr = pd.read_csv("rand_moments_corr.csv")

best_df = df[df["file"].str.contains("23")]

moment_columns = [col for col in df.columns if col.startswith("H") and not col.endswith("err")]
moment_columns_err = [col for col in df.columns if col.startswith("H") and col.endswith("err")]
print(moment_columns)

## Analysis

### Histogram of moment fit results
Lets take a quick look at a scatter plot of each moment for the 100 fits, vs its corresponding likelihood

In [ ]:
import matplotlib.pyplot as plt

for column in moment_columns:
    alpha = column[1]
    Jv, Lambda, J, M = list(column.split("_")[1])
    x_label = rf"$H^{alpha}({Jv},{Lambda},{J},{M})$"
    
    min_likelihood = df['likelihood'].min()
    error_column = f"{column}_err"  # Corresponding error column
    
    # Define x-axis limits based on the data range
    x_min = df[column].min() - abs(df[column].min()*0.1)  # Add some padding
    x_max = df[column].max() + abs(df[column].max()*0.1)  # Add some padding
    
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    
    # First subplot: Original scatter plot with error bars
    axes[0].errorbar(df[column], df['likelihood'] - min_likelihood, xerr=df[error_column], fmt='o', alpha=0.7, label="Fits")
    axes[0].errorbar(best_df[column], best_df['likelihood'] - min_likelihood, xerr=best_df[error_column], fmt='o', color='green', markersize=10, label="Best Fit")
    axes[0].set_xlim(x_min, x_max)  # Set x-axis limits
    axes[0].set_xlabel(x_label, loc="right", size=14)
    axes[0].set_ylabel('Δ Likelihood', loc="top", size=14)
    axes[0].grid(True)
    axes[0].set_title("Δ Likelihood vs. Moment")
    axes[0].legend()
    
    # Second subplot: Scatter plot with adjusted y-limits and error bars
    axes[1].errorbar(df[column], df['likelihood'] - min_likelihood, xerr=df[error_column], fmt='o', alpha=0.7, label="Fits")
    axes[1].errorbar(best_df[column], best_df['likelihood'] - min_likelihood, xerr=best_df[error_column], fmt='o', color='green', markersize=10, label="Best Fit")
    axes[1].set_xlim(x_min, x_max)  # Set x-axis limits
    axes[1].set_xlabel(x_label, loc="right", size=14)
    axes[1].set_ylabel('Δ Likelihood', loc="top", size=14)
    axes[1].grid(True)
    axes[1].set_ylim(0 - 1, 10)
    axes[1].set_title("Zoomed into Best Fit Region")
    axes[1].legend()
    
    plt.tight_layout()
    plt.show()

### $\chi^2$ vs Likelihood
To get a sense of how different our results were for each randomized fit, we can calculate a $\chi^2$ for the moment parameters, and compare this to likelihood. We would expect to find our best result is different from the randomized fits.

In [ ]:
# Initialize a list to store chi2 values
chi2_values = []

# Iterate over each row in df
for index, row in df.iterrows():
    chi2 = 0
    for moment, moment_err in zip(moment_columns, moment_columns_err):
        observed = row[moment]
        expected = best_df[moment].values[0]  # Best fit value for the moment
        error = row[moment_err]
        
        # Avoid division by zero
        if error > 0:
            chi2 += ((observed - expected) ** 2) / (error ** 2)
    ndf = len(moment_columns)  # Number of degrees of freedom
    chi2_values.append(chi2 / ndf)  # Store the reduced chi2 value

# Add the chi2 values as a new column in the DataFrame
df['chi2'] = chi2_values

import matplotlib.pyplot as plt

# Scatter plot of chi2 vs Δ likelihood
min_likelihood = best_df["likelihood"].iloc[0]

plt.figure(figsize=(10, 6))
plt.scatter(df['chi2'], df['likelihood'] - min_likelihood, alpha=0.7, label="Fits")

# Add labels and title
plt.xlabel(r"$\chi^2$", fontsize=14)
plt.ylabel("Δ Likelihood", fontsize=14)
plt.grid(True)
plt.legend()
plt.tight_layout()

# Show the plot
plt.show()

# repeat, just on log-log scale
plt.figure(figsize=(10, 6))
plt.scatter(df['chi2'], df['likelihood'] - min_likelihood, alpha=0.7, label="Fits")

# Add labels and title
plt.xlabel(r"$\chi^2/ndf$", fontsize=14)
plt.ylabel("Δ Likelihood", fontsize=14)
plt.grid(True)
plt.legend()
plt.yscale('log')
plt.xscale('log')
plt.tight_layout()

# Show the plot
plt.show()

### Correlations

#### Best fit Correlations

#### Consistently large correlations across randomized fits